# SeQuant benchmarking notebook
This notebook has been written for different embeddings strategies comparison on several benchmark datasets

# Data preprocessing

## Libs import

In [1]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 75.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import requests
import os
import json
import time

from sklearn.preprocessing import OneHotEncoder
from Bio.Align import substitution_matrices
from itertools import product

## Benchmark datasets import

In [3]:
antioxidative = pd.read_csv('antiox.csv')
antiinflamatory = pd.read_csv('antiinf.csv')
antimicrobial = pd.read_csv('antimic.csv')
antidiabetic = pd.read_csv('antidia.csv')

benchmark_list = [antioxidative, antiinflamatory, antimicrobial, antidiabetic]

In [4]:
for dataset in benchmark_list:
  dataset['length'] = dataset['seq'].apply(len)
  length = dataset['length'].max()
  print(length)


20
30
30
41


## Sequence encoding

### Encoding functions

In [5]:
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

In [7]:
def one_hot_encode(sequence):
    encoder = OneHotEncoder(categories=[list(amino_acids)], dtype=int, sparse_output=False)
    sequence_array = np.array(list(sequence)).reshape(-1, 1)
    encoded = encoder.fit_transform(sequence_array).flatten()
    return encoded


def threemers_encode(sequence):
    k = 3
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]

    kmer_to_index = {kmer: idx for idx, kmer in enumerate([''.join(p) for p in product(amino_acids, repeat=k)])}
    encoded = [kmer_to_index[kmer] for kmer in kmers]
    return encoded


def blosum62_encode(sequence):
    blosum62 = substitution_matrices.load("BLOSUM62")
    encoded_vector = []
    for i in range(len(sequence) - 1):
        pair = (sequence[i], sequence[i+1])
        if pair in blosum62:
            encoded_vector.append(blosum62[pair])
        elif (pair[1], pair[0]) in blosum62:
            encoded_vector.append(blosum62[(pair[1], pair[0])])
        else:
            encoded_vector.append(0)
    return encoded_vector


def process_dataset(df, encoding_func, encoding_name, pad_value):
    encoded_data = df['seq'].apply(encoding_func)
    max_len = max(encoded_data.apply(len))

    encoded_data = encoded_data.apply(lambda x: np.pad(x, (0, max_len - len(x)), 'constant', constant_values=pad_value))

    encoded_df = pd.DataFrame(encoded_data.tolist(), index=df.index)

    result_df = pd.concat([df, encoded_df], axis=1)

    return result_df



### Functions implementation

In [8]:
datasets = ['antimic', 'antidia', 'antiinf', 'antiox']
for dataset in datasets:
    df = pd.read_csv(f'{dataset}.csv')

    # One-hot encoding with padding 20
    one_hot_df = process_dataset(df, one_hot_encode, 'one_hot', pad_value=0)
    one_hot_df.to_csv(f'{dataset}_one_hot.csv', index=False)

    # Threemers encoding with padding of max threemer index + 1
    threemers_pad_value = len('ACDEFGHIKLMNPQRSTVWY') ** 3
    threemers_df = process_dataset(df, threemers_encode, 'threemers', pad_value=threemers_pad_value)
    threemers_df.to_csv(f'{dataset}_threemers.csv', index=False)

    # BLOSUM62 encoding with padding 0
    blosum62_df = process_dataset(df, blosum62_encode, 'blosum62', pad_value=0)
    blosum62_df.to_csv(f'{dataset}_blosum62.csv', index=False)

### SeQuant API usage for SeQuant embeddings

In [ ]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

for dataset in datasets:
    df = pd.read_csv(f'{dataset}.csv')
    sequences = list(df['seq'])

    several_id_lists = np.array_split(np.asarray(sequences), int(len(sequences) / 50) + 1)

    df_fin_data = pd.DataFrame()

    for i in several_id_lists:
        params = {
            'sequences': ', '.join(list(i)),
            'polymer_type': 'protein',
            'encoding_strategy': 'protein',
            'skip_unprocessable': 'true',
        }

        time.sleep(1)
        response = requests.post('https://ai-chemistry.itmo.ru/api/encode_sequence', params=params, headers=headers)
        assert response.status_code == 200
        a = json.loads(response.content)
        data = pd.DataFrame.from_dict(a, orient='index')
        df_fin_data = pd.concat([df_fin_data, data])

    df_fin_data['seq'] = df_fin_data.index

    final_df = pd.merge(df, df_fin_data, on='seq', how='inner')

    final_df.to_csv(f'{dataset}_sequant.csv', index=False)


# XGB classifier learning



## Libs import

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, confusion_matrix
from xgboost import XGBClassifier
import joblib

## Model learning

In [ ]:
tasks = ['antimic', 'antidia', 'antiinf', 'antiox']
encoding_strategies = ['one_hot', 'threemers', 'blosum62', 'sequant']

In [ ]:
metrics_list = []

for task in tasks:
    for encoding in encoding_strategies:

        filename = f"{task}_{encoding}.csv"
        df = pd.read_csv(filename)

        descriptors = df.drop(columns=['seq', 'label'])

        scaler = MinMaxScaler()
        scaled_descriptors = scaler.fit_transform(descriptors)

        # Save scaler
        joblib.dump(scaler, f"{task}_{encoding}_scaler.pkl")

        scaled_df = pd.DataFrame(scaled_descriptors, columns=descriptors.columns)
        scaled_df['seq'] = df['seq']
        scaled_df['label'] = df['label']

        # Train_test split with stratification on label and random state = 11
        train_df, test_df = train_test_split(scaled_df, test_size=0.2, stratify=scaled_df['label'], random_state=11)

        model = XGBClassifier()

        X_train = train_df.drop(columns=['seq', 'label'])
        y_train = train_df['label']

        # 5-fold cross validation
        y_pred_cv = cross_val_predict(model, X_train, y_train, cv=5)

        # Estimation on test
        X_test = test_df.drop(columns=['seq', 'label'])
        y_test = test_df['label']

        model.fit(X_train, y_train)
        y_pred_test = model.predict(X_test)

        # Evaluation
        accuracy = accuracy_score(y_test, y_pred_test)
        precision = precision_score(y_test, y_pred_test)
        recall = recall_score(y_test, y_pred_test)
        f1 = f1_score(y_test, y_pred_test)
        roc_auc = roc_auc_score(y_test, y_pred_test)
        mcc = matthews_corrcoef(y_test, y_pred_test)

        cv_accuracy = accuracy_score(y_train, y_pred_cv)
        cv_precision = precision_score(y_train, y_pred_cv)
        cv_recall = recall_score(y_train, y_pred_cv)
        cv_f1 = f1_score(y_train, y_pred_cv)
        cv_roc_auc = roc_auc_score(y_train, y_pred_cv)
        cv_mcc = matthews_corrcoef(y_train, y_pred_cv)

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()

        # Save model
        model_filename = f"{task}_{encoding}_model.pkl"
        joblib.dump(model, model_filename)

        metrics_list.append({
            'task': task,
            'encoding_strategy': encoding,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'roc_auc': roc_auc,
            'mcc': mcc,
            'cv_accuracy': cv_accuracy,
            'cv_precision': cv_precision,
            'cv_recall': cv_recall,
            'cv_f1_score': cv_f1,
            'cv_roc_auc': cv_roc_auc,
            'cv_mcc': cv_mcc,
            'tn': tn,
            'fp': fp,
            'fn': fn,
            'tp': tp
        })

# Create metrics df
metrics_df = pd.DataFrame(metrics_list)
metrics_df.to_csv("model_metrics.csv", index=False)